In [2]:
%load_ext blackcellmagic
%matplotlib inline

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [3]:
from matplotlib import style

style.use("fivethirtyeight")
import matplotlib.pyplot as plt

In [4]:
import numpy as np
import pandas as pd

In [5]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [6]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [7]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
inspector = inspect(engine)

In [8]:
table_names = inspector.get_table_names()
table_names

In [9]:
for table in table_names:
    print(table + ":")
    columns = inspector.get_columns(table)
    column_names = [column["name"] for column in columns]
    print(column_names)

In [10]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [11]:
# We can view all of the classes that automap found
Base.classes.keys()

In [12]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [13]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

* Design a query to retrieve the last 12 months of precipitation data and plot the results

* Calculate the date 1 year ago from the last data point in the database

* Perform a query to retrieve the data and precipitation scores

* Save the query results as a Pandas DataFrame and set the index to the date column

* Sort the dataframe by date

* Use Pandas Plotting with Matplotlib to plot the data

In [14]:
session.query(Measurement.date).count()

In [15]:
print(engine.execute("SELECT * FROM measurement ORDER BY date DESC").first())

In [16]:
last_data_point = (
    session.query(Measurement.date).order_by(Measurement.date.desc()).first().date
)

In [17]:
last_data_point

In [18]:
# last_year = dt.datetime.strptime(last_data_point, "%Y-%m-%d") - dt.timedelta(days=365)

In [19]:
last_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print(last_year)

In [20]:
prcp_last_year = (
    session.query(Measurement.date, Measurement.prcp)
    .filter(Measurement.date >= last_year)
    .order_by(Measurement.date)
    .all()
)

In [21]:
prcp_last_year_df = pd.DataFrame(prcp_last_year).set_index("date")

In [22]:
prcp_last_year_df.head()

In [23]:
prcp_last_year_df.plot(y="prcp", use_index=True, label="Precipitation")
plt.gcf().set_size_inches(20, 10)
plt.xlabel("Date")
plt.xticks(rotation=45)
plt.ylabel("Amount of Precipitation")
plt.title("Precipitation Analysis")
plt.legend()
plt.show()
plt.close()

In [24]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_last_year_df.describe()

# Station Analysis

* Design a query to calculate the total number of stations.

* Design a query to find the most active stations.

* List the stations and observation counts in descending order.

* Which station has the highest number of observations?

* Hint: You may need to use functions such as `func.min`, `func.max`, `func.avg`, and `func.count` in your queries.

* Design a query to retrieve the last 12 months of temperature observation data (tobs).

* Filter by the station with the highest number of observations.

* Plot the results as a histogram with `bins=12`.

In [25]:
# Design a query to show how many stations are available in this dataset?
number_stations = session.query(Station.station).count()
print(number_stations)

In [26]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
active_stations = (
    session.query(Measurement.station, func.count(Measurement.station))
    .group_by(Measurement.station)
    .order_by(func.count(Measurement.station).desc())
    .all()
)
active_stations

In [27]:
# Using the station id from the previous query, calculate the lowest temperature recorded,
# highest temperature recorded, and average temperature most active station?
most_active_station = (
    session.query(
        func.min(Measurement.tobs),
        func.max(Measurement.tobs),
        func.avg(Measurement.tobs),
    )
    .filter(Measurement.station == "USC00519281")
    .first()
)

In [28]:
most_active_station

In [29]:
# Choose the station with the highest number of temperature observations.
station_most_temps = (
    session.query(Measurement.station, func.count(Measurement.tobs))
    .group_by(Measurement.station)
    .order_by(func.count(Measurement.station).desc())
    .first()
)
print(station_most_temps)

In [30]:
# Query the last 12 months of temperature observation data for this station
temp_last_year = (
    session.query(Measurement.station, Measurement.tobs)
    .filter(Measurement.date >= last_year)
    .filter(Measurement.station == "USC00519281")
    .all()
)

In [31]:
temp_last_year_df = pd.DataFrame(temp_last_year)
temp_last_year_df

In [32]:
# plot the results as a histogram
temp_last_year_df.plot.hist(bins=12,)
plt.gcf().set_size_inches(10, 8)
plt.xlabel("Temperature")
plt.title("Station USC00519281 Temperature Histogram")
plt.legend(["Temperature"])
plt.show()
plt.close()

In [33]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d'
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """

    return (
        session.query(
            func.min(Measurement.tobs),
            func.avg(Measurement.tobs),
            func.max(Measurement.tobs),
        )
        .filter(Measurement.date >= start_date)
        .filter(Measurement.date <= end_date)
        .all()
    )


# function usage example
print(calc_temps("2012-02-28", "2012-03-05"))